# Use AutoGen in Microsoft Fabric

AutoGen offers conversable LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

[Microsoft Fabric](https://learn.microsoft.com/en-us/fabric/get-started/microsoft-fabric-overview) is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place. Its pre-built AI models include GPT-x models such as `gpt-4-turbo`, `gpt-4`, `gpt-4-8k`, `gpt-4-32k`, `gpt-35-turbo`, `gpt-35-turbo-16k` and `gpt-35-turbo-instruct`.

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform a task which require acquiring info from the web:
* discuss a paper based on its URL.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen:
```bash
pip install pyautogen
```

Also, this notebook depends on Microsoft Fabric pre-built LLM endpoints. Running it elsewhere may encounter errors.

In [ ]:
%pip install pyautogen -q

StatementMeta(, , -1, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Set your API endpoint

In [ ]:
from synapse.ml.mlflow import get_mlflow_env_config

mlflow_env_configs = get_mlflow_env_config()
access_token = mlflow_env_configs.driver_aad_token
prebuilt_AI_base_url = mlflow_env_configs.workload_endpoint + "cognitive/openai/"

StatementMeta(, 339dbc6f-779f-40b8-a484-08b1cda777f3, 9, Finished, Available)

2023-12-09:08:45:47,911 WARNING  [synapse_mlflow_utils.py:244] To save or load Apache Spark model files, please attach a Lakehouse.


In [ ]:
config_list = [
    {
        'model': 'gpt-4',
        'api_key': access_token,
        'base_url': prebuilt_AI_base_url,
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
]


llm_config={
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

StatementMeta(, 339dbc6f-779f-40b8-a484-08b1cda777f3, 10, Finished, Available)

## Construct Agents

We construct the assistant agent and the user proxy agent.

In [ ]:
import autogen

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",  # input() doesn't work, so needs to be "NEVER" here
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

StatementMeta(, 339dbc6f-779f-40b8-a484-08b1cda777f3, 11, Finished, Available)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation.

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

StatementMeta(, 339dbc6f-779f-40b8-a484-08b1cda777f3, 12, Finished, Available)

user_proxy (to assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------
assistant (to user_proxy):

To determine who should read the paper, we first need to understand the content of the paper. We can do this by fetching the abstract of the paper from the provided URL and analyzing it. 

Here is a Python script that uses the BeautifulSoup library to scrape the abstract from the webpage.

```python
# filename: fetch_abstract.py

import requests
from bs4 import BeautifulSoup

def fetch_abstract(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    abstract = soup.find('blockquote', attrs={'class': 'abstract mathjax'}).text.strip()
    return abstract

url = "https://arxiv.org/abs/2308.08155"
print(fetch_abstract(url))
```

Please save this script as `fetch_abstract.py` and run it. This will print the abstract of the paper. Based on the abstract,